#### **Keras Neural Network**

In [ ]:
tf.random.set_seed(0)

In [ ]:
def createModel_tuner(hp):
    model = Sequential()
    for i in range(hp.Int('num_layers', 2, 10)):
        model.add(Dense(units=hp.Int('units_' + str(i),
                                        min_value=32,
                                        max_value=512,
                                        step=32),
                            activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(
        optimizer=hp.Choice('optimizer', ['adam', 'sgd']),
        loss='binary_crossentropy',
        metrics=['accuracy'])
    return model

In [ ]:
tuner = RandomSearch(
    createModel_tuner,
    objective='val_accuracy',
    max_trials=10,
    directory='project',
    project_name='suffer')

tuner.search_space_summary()

Search space summary
Default search space size: 4
num_layers (Int)
{'default': None, 'conditions': [], 'min_value': 2, 'max_value': 10, 'step': 1, 'sampling': None}
units_0 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}
units_1 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}
optimizer (Choice)
{'default': 'adam', 'conditions': [], 'values': ['adam', 'sgd'], 'ordered': False}


In [ ]:
tuner.search(x_train, y_train, epochs=70, batch_size=32, validation_data=(x_test, y_test))

Trial 7 Complete [00h 13m 32s]
val_accuracy: 0.8962233066558838

Best val_accuracy So Far: 0.8966662883758545
Total elapsed time: 01h 23m 00s

Search: Running Trial #8

Value             |Best Value So Far |Hyperparameter
6                 |3                 |num_layers
416               |128               |units_0
256               |480               |units_1
sgd               |adam              |optimizer
64                |320               |units_2
160               |352               |units_3
320               |192               |units_4
480               |320               |units_5
224               |480               |units_6
160               |384               |units_7
128               |320               |units_8

Epoch 1/70
1129/1129 [==============================] - 15s 12ms/step - loss: 0.3840 - accuracy: 0.8825 - val_loss: 0.3445 - val_accuracy: 0.8835
Epoch 2/70
1129/1129 [==============================] - 11s 10ms/step - loss: 0.3430 - accuracy: 0.8833 - val_loss: 0.34

In [ ]:
tuner.results_summary(num_trials=5)

In [ ]:
kerasNN_tuner_pred = (tuner.get_best_models()[0].predict(x_test)).round().astype(int)
print("Classification report on Test data with Keras NN Model with Adam Optimizer\n=======================")
print(classification_report(y_true=y_test, y_pred=kerasNN_tuner_pred))

In [ ]:
def createModel_tuner2(hp):
    model = Sequential()
    for i in range(hp.Int('num_layers', 2, 10)):
        model.add(Dense(units=hp.Int('units_' + str(i),
                                        min_value=16,
                                        max_value=512,
                                        step=32),
                            activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(
        optimizer=Adam(learning_rate=hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='LOG', default=1e-3)),
        loss='binary_crossentropy',
        metrics=['accuracy'])
    return model

In [ ]:
tunerAdam = RandomSearch(
    createModel_tuner2,
    objective='val_accuracy',
    max_trials=5,
    directory='project',
    project_name='sufferAdam')

tunerAdam.search_space_summary()

In [ ]:
tunerAdam.search(x_train, y_train, epochs=70, batch_size=32, validation_data=(x_test, y_test))

In [ ]:
tunerAdam.results_summary(num_trials=3)

In [ ]:
kerasNN_adam_pred = (tunerAdam.get_best_models()[0].predict(x_test)).round().astype(int)
print("Classification report on Test data with Keras NN Model with Adam Optimizer\n=======================")
print(classification_report(y_true=y_test, y_pred=kerasNN_adam_pred))

In [ ]:
def createModel_tuned(IL, HL):
    model = Sequential(name="Tuned_Model")
    model.add(Dense(IL, input_shape=(x_train.shape[1],), activation='relu', name="Input_Layer"))
    for c, i in enumerate(HL):
        model.add(Dense(units=i, activation='relu', name=f'Hidden_Layer_{c}'))
    model.add(Dense(1, activation='sigmoid', name="Output_Layer"))
    model.compile(
        optimizer=Adam(learning_rate=0.0001),
        loss='binary_crossentropy',
        metrics=['accuracy'])
    return model

In [ ]:
tuned_model = createModel_tuned(512, [448, 384, 320, 256, 192, 128, 64, 32, 16, 8])
tuned_model.summary()

In [ ]:
history_tuned = tuned_model.fit(x_train, y_train, epochs=70, batch_size=32, validation_data=(x_test, y_test), use_multiprocessing=True)

In [ ]:
# Get training and test loss histories
training_loss = history_tuned.history['loss']
test_loss = history_tuned.history['val_loss']
training_acc = history_tuned.history['accuracy']
test_acc = history_tuned.history['val_accuracy']

# Create count of the number of epochs
epoch_count = range(1, len(training_loss) + 1)

# Visualize loss history
plt.figure(figsize=(24, 12))
plt.subplot(1, 2, 1)
plt.plot(epoch_count, training_loss, 'r--')
plt.plot(epoch_count, test_loss, 'b-')
plt.legend(['Training Loss', 'Test Loss'])
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.subplot(1, 2, 2)
plt.plot(epoch_count, training_acc, 'r--')
plt.plot(epoch_count, test_acc, 'b-')
plt.legend(['Training Accuracy', 'Test Accuracy'])
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.show()

In [ ]:
kerasNN_tuned_pred = (tuned_model.predict(x_test)).round().astype(int)
print("Classification report on Test data with Keras NN Tuned Model\n=======================")
print(classification_report(y_true=y_test, y_pred=kerasNN_tuned_pred))

In [ ]:
tuned_model2 = createModel_tuned(64, [16])
tuned_model2.summary()

In [ ]:
history_tuned2 = tuned_model2.fit(x_train, y_train, epochs=70, batch_size=32, validation_data=(x_test, y_test), use_multiprocessing=True)

In [ ]:
# Get training and test loss histories
training_loss = history_tuned2.history['loss']
test_loss = history_tuned2.history['val_loss']
training_acc = history_tuned2.history['accuracy']
test_acc = history_tuned2.history['val_accuracy']

# Create count of the number of epochs
epoch_count = range(1, len(training_loss) + 1)

# Visualize loss history
plt.figure(figsize=(24, 12))
plt.subplot(1, 2, 1)
plt.plot(epoch_count, training_loss, 'r--')
plt.plot(epoch_count, test_loss, 'b-')
plt.legend(['Training Loss', 'Test Loss'])
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.subplot(1, 2, 2)
plt.plot(epoch_count, training_acc, 'r--')
plt.plot(epoch_count, test_acc, 'b-')
plt.legend(['Training Accuracy', 'Test Accuracy'])
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.show()

In [ ]:
kerasNN_tuned2_pred = (tuned_model2.predict(x_test)).round().astype(int)
print("Classification report on Test data with Keras NN Tuned Model 2\n=======================")
print(classification_report(y_true=y_test, y_pred=kerasNN_tuned2_pred))